!pip install grpcio==1.42.0 tensorflow-serving-api==2.7.0

In [63]:
import grpc
import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

from io import BytesIO
from urllib import request
import numpy as np
from PIL import Image

In [64]:
host = 'localhost:8500'

channel = grpc.insecure_channel(host , options=(('grpc.enable_http_proxy', 0),))

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)


In [65]:
stub

In [66]:
url = "http://bit.ly/mlbookcamp-pants"

In [67]:
classes = ['dress',
        'hat',
        'longsleeve',
        'outwear',
        'pants',
        'shirt',
        'shoes',
        'shorts',
        'skirt',
        't-shirt']

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

def preprocess_input(url):
    x = download_image(url)
    x = x.resize((299, 299), Image.NEAREST)
    arr = np.array(x, dtype="float32")
    arr = np.array([arr])
    arr /= 127.5
    arr -= 1.
    return arr

def decode_result(pred):
    result = {c: float(p) for c, p in zip(classes, pred)}
    return result


In [68]:
X = preprocess_input(url)

C:\Users\soumy\AppData\Local\Temp\ipykernel_6156\2201985149.py:21: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  x = x.resize((299, 299), Image.NEAREST)


In [69]:
def numpy_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [72]:
pb_request = predict_pb2.PredictRequest()

#   from model-description.txt
pb_request.model_spec.name = 'clothing-model'
pb_request.model_spec.signature_name = 'serving_default'
pb_request.inputs['input_28'].CopyFrom(numpy_to_protobuf(X))

In [74]:
pb_response = stub.Predict(pb_request, timeout=20.0)
pb_response

outputs {
  key: "dense_18"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 1
      }
      dim {
        size: 10
      }
    }
    float_val: -2.355492353439331
    float_val: -5.121376991271973
    float_val: -0.9679684638977051
    float_val: -0.9708386659622192
    float_val: 8.349003791809082
    float_val: -1.6028835773468018
    float_val: -4.878284931182861
    float_val: 3.3535468578338623
    float_val: -1.9602851867675781
    float_val: -4.606760025024414
  }
}
model_spec {
  name: "clothing-model"
  version {
    value: 1
  }
  signature_name: "serving_default"
}

In [78]:
preds = pb_response.outputs["dense_18"].float_val

In [79]:
decode_result(preds)

{'dress': -2.355492353439331,
 'hat': -5.121376991271973,
 'longsleeve': -0.9679684638977051,
 'outwear': -0.9708386659622192,
 'pants': 8.349003791809082,
 'shirt': -1.6028835773468018,
 'shoes': -4.878284931182861,
 'shorts': 3.3535468578338623,
 'skirt': -1.9602851867675781,
 't-shirt': -4.606760025024414}